## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-02-07-13-41 +0000,bbc,Stars hit red carpet at Grammy Awards show,https://www.bbc.com/news/articles/cqj2kyp29l7o...
1,2026-02-02-07-07-00 +0000,wsj,"U.S. Treasury Yields Fall Ahead of Data, Refun...",https://www.wsj.com/finance/investing/jgbs-fal...
2,2026-02-02-07-00-55 +0000,bbc,Twelve miners killed by Russian strike in Ukra...,https://www.bbc.com/news/articles/cdre7g2je63o...
3,2026-02-02-06-57-00 +0000,wsj,Julius Baer Posts Expected Drop in Profit,https://www.wsj.com/finance/banking/julius-bae...
4,2026-02-02-06-51-45 +0000,nyt,Laura Fernández Wins Costa Rican Presidency,https://www.nytimes.com/2026/02/01/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2449/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
163,trump,23
60,epstein,19
94,ice,15
34,new,11
152,iran,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
61,2026-02-02-00-53-49 +0000,nypost,Trump announces two-year closure of Trump Kenn...,https://nypost.com/2026/02/01/us-news/trump-an...,76
84,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...,60
40,2026-02-02-02-56-44 +0000,nyt,Trump Says Kennedy Center Will Close for 2-Yea...,https://www.nytimes.com/2026/02/01/us/politics...,56
43,2026-02-02-02-49-20 +0000,nypost,"5-year-old boy, father taken into custody by I...",https://nypost.com/2026/02/01/us-news/5-year-o...,56
98,2026-02-01-21-36-04 +0000,nyt,How Trump Appears in the Epstein Files,https://www.nytimes.com/2026/02/01/us/trump-ep...,51


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
61,76,2026-02-02-00-53-49 +0000,nypost,Trump announces two-year closure of Trump Kenn...,https://nypost.com/2026/02/01/us-news/trump-an...
170,50,2026-02-01-13-52-32 +0000,nypost,Ex-Prince Andrew’s creepy reply to Jeffrey Eps...,https://nypost.com/2026/02/01/world-news/ex-pr...
43,46,2026-02-02-02-49-20 +0000,nypost,"5-year-old boy, father taken into custody by I...",https://nypost.com/2026/02/01/us-news/5-year-o...
114,40,2026-02-01-20-12-26 +0000,nypost,Gov. Hochul will raise New York’s taxes after ...,https://nypost.com/2026/02/01/us-news/cowardly...
192,34,2026-02-01-10-54-14 +0000,nypost,Iran’s supreme leader warns any US attack woul...,https://nypost.com/2026/02/01/world-news/iran-...
84,26,2026-02-01-22-34-48 +0000,nypost,India to buy oil from Venezuela rather than Ru...,https://nypost.com/2026/02/01/world-news/india...
89,25,2026-02-01-22-15-31 +0000,bbc,'Do you think you're the devil himself?' Epste...,https://www.bbc.com/news/videos/cx20zqw7v9ko?a...
88,24,2026-02-01-22-21-00 +0000,wsj,A state-level Democratic win in a region that ...,https://www.wsj.com/politics/elections/texas-e...
99,24,2026-02-01-21-34-42 +0000,nypost,"PENN District poised to grow by another block,...",https://nypost.com/2026/02/01/business/penn-di...
54,23,2026-02-02-01-31-39 +0000,nypost,Mike Johnson ‘confident’ gov’t shutdown will e...,https://nypost.com/2026/02/01/us-news/mike-joh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
